In [1]:
import pandas as pd
import io
import boto3
import numpy as np

In [5]:
municipios = pd.read_csv("s3://g01batalha/mun_pop.csv")
municipios_desc = pd.read_csv("s3://g01batalha/mun_desc.csv")
municipios_desc = municipios_desc.rename(columns={'Codmun': 'Código IBGE do Município'})
municipios_info = pd.merge(municipios, municipios_desc,  how='inner', on=['Código IBGE do Município'])
municipios_info = municipios_info[['NomeMunic','Sigla da Unidade da Federação','Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)', 'PIB per capita a preços correntes', 'População residente','População residente - Mulheres']]

In [76]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
municipios_info.head()

,NomeMunic,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres
0,ALTA FLORESTA D'OESTE,RO,0.641,13743.50,24392,11736
1,ARIQUEMES,RO,0.702,14315.20,90353,44810
2,CABIXI,RO,0.650,15755.15,6313,3047
3,CACOAL,RO,0.718,14865.49,78574,39450
4,CEREJEIRAS,RO,0.692,15996.62,17029,8478


In [22]:
municipios_info = municipios_info.loc[(municipios_info['Sigla da Unidade da Federação'] == "SP") & (municipios_info['População residente'] >= 400000)]

In [10]:
data = {'NomeMunic': ['SÃO PAULO', 'SÃO JOSÉ DO RIO PRETO', 'GUARULHOS', 'MAUÁ','SANTOS','CAMPINAS','SÃO BERNARDO DO CAMPO','OSASCO','SANTO ANDRÉ','SÃO JOSÉ DOS CAMPOS', 'SOROCABA','RIBEIRÃO PRETO'], 'coeficiente_feminicidio': [2.2, 2.4, 3.2, 3.9, 1.8, 3.0, 1.2, 2.2, 1.4,1.5,2.7,1.8]}
feminicidio = pd.DataFrame.from_dict(data)
##Source https://www.scielosp.org/article/csc/2017.v22n9/2963-2970/#

In [15]:
df_idade_homicidios = pd.read_excel("s3://batalhadedadositau/seguranca/ITAU/ViolenciaContraMulher/BASE 13 - Feminicidio Por UF e Cor/Homicidios por Idade e genero em 2014.xlsx")

In [16]:
df_idade_homicidios_mulher = df_idade_homicidios[['idade','Feminino']]

In [23]:
municipios_info['tmp'] = 1
df_idade_homicidios_mulher['tmp'] = 1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_merge = pd.merge(df_idade_homicidios_mulher, municipios_info, on=['tmp'])
df_merge = df_merge.drop('tmp', axis=1)

In [25]:
df_merge.loc[df_merge['idade'] == "<1", 'idade'] = 0

In [26]:
df_merge

,idade,Feminino,NomeMunic,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres
0,0,1.5,CAMPINAS,SP,0.805,33939.56,1080113,559248
1,0,1.5,GUARULHOS,SP,0.763,30383.43,1221979,626936
2,0,1.5,MAUÁ,SP,0.766,17619.04,417064,212971
3,0,1.5,OSASCO,SP,0.776,54599.81,666740,346304
4,0,1.5,RIBEIRÃO PRETO,SP,0.800,28100.52,604682,314511
5,0,1.5,SANTO ANDRÉ,SP,0.815,25609.30,676407,351949
6,0,1.5,SANTOS,SP,0.840,65790.53,419400,227488
7,0,1.5,SÃO BERNARDO DO CAMPO,SP,0.805,46495.62,765463,395837
8,0,1.5,SÃO JOSÉ DO RIO PRETO,SP,0.797,21991.26,408258,212242
9,0,1.5,SÃO JOSÉ DOS CAMPOS,SP,0.807,38431.00,629921,321297


In [93]:
df_merge_calc_coef = pd.merge(municipios_info, feminicidio,  how='inner', on=['NomeMunic'])

In [94]:
df_merge_calc_coef

,NomeMunic,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres,tmp,coeficiente_feminicidio
0,CAMPINAS,SP,0.805,33939.56,1080113,559248,1,3.0
1,GUARULHOS,SP,0.763,30383.43,1221979,626936,1,3.2
2,MAUÁ,SP,0.766,17619.04,417064,212971,1,3.9
3,OSASCO,SP,0.776,54599.81,666740,346304,1,2.2
4,RIBEIRÃO PRETO,SP,0.800,28100.52,604682,314511,1,1.8
5,SANTO ANDRÉ,SP,0.815,25609.30,676407,351949,1,1.4
6,SANTOS,SP,0.840,65790.53,419400,227488,1,1.8
7,SÃO BERNARDO DO CAMPO,SP,0.805,46495.62,765463,395837,1,1.2
8,SÃO JOSÉ DO RIO PRETO,SP,0.797,21991.26,408258,212242,1,2.4
9,SÃO JOSÉ DOS CAMPOS,SP,0.807,38431.00,629921,321297,1,1.5


In [ ]:
df_merge_calc_coef = df_merge_calc_coef

In [96]:
Total = df_merge_calc_coef['coeficiente_feminicidio'].sum()

In [97]:
Total

27.299999999999997

In [27]:
df_merge_final = pd.merge(df_merge, feminicidio,  how='inner', on=['NomeMunic'])

In [28]:
df_merge_final

,idade,Feminino,NomeMunic,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres,coeficiente_feminicidio
0,0,1.5,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
1,1,0.4,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
2,2,0.3,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
3,3,0.2,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
4,4,0.1,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
5,5,0.2,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
6,6,0.2,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
7,7,0.1,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
8,8,0.2,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0
9,9,0.1,CAMPINAS,SP,0.805,33939.56,1080113,559248,3.0


In [77]:
df_merge_final.shape

(972, 9)

In [29]:
df_BO_2007 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2007/data/clean_2007.csv")
df_BO_2008 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2008/data/clean_2008.csv")
df_BO_2009 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2009/data/clean_2009.csv")
df_BO_2010 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2010/data/clean_2010.csv")
df_BO_2011 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2011/data/clean_2011.csv")
df_BO_2012 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2012/data/clean_2012.csv")
df_BO_2013 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2013/data/clean_2013.csv")
df_BO_2014 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2014/data/clean_2014.csv")
df_BO_2015 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2015/data/clean_2015.csv")
df_BO_2016 = pd.read_csv("s3://batalhadedadositau/seguranca/ITAU/CidadeSegura/Base 08 - Transparência Dados BO/BOs2007a2016/crime-data-in-brazil/boletins-de-ocorrencia-sp-2016/data/clean_2016.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17,18,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,17,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df_BO_2007a2016 = pd.concat([df_BO_2007,df_BO_2008,df_BO_2009,df_BO_2010,df_BO_2011,df_BO_2012,df_BO_2013])

In [31]:
df_BO_2007a2016 = df_BO_2007a2016.drop_duplicates(['num_bo','ano_bo','id_delegacia']).reset_index()
df_BO_2007a2016 = df_BO_2007a2016.loc[df_BO_2007a2016['descr_tipo_pessoa'] == "Vítima"]
df_BO_2007a2016 = df_BO_2007a2016.loc[df_BO_2007a2016['sexo_pessoa'] == "F"]

In [32]:
df_BO_2007a2016_homicidio = df_BO_2007a2016.loc[(df_BO_2007a2016['rubrica'] == "Homicídio culposo (art. 121, §3o.)") | (df_BO_2007a2016['rubrica'] == "Homicídio qualificado (art. 121, §2o.)") | (df_BO_2007a2016['rubrica'] == "Homicídio simples (art. 121)")]
df_BO_2007a2016_lesao = df_BO_2007a2016.loc[(df_BO_2007a2016['rubrica'] == "Lesão corporal de natureza 'GRAVÍSSIMA' (art. 129, §2o.)") | (df_BO_2007a2016['rubrica'] == "Lesão corporal (art 129 § 9º)") | (df_BO_2007a2016['rubrica'] == "Lesão corporal (art. 129)") | (df_BO_2007a2016['rubrica'] == "Lesão corporal culposa (art. 129. §6o.)") | (df_BO_2007a2016['rubrica'] == "Lesão corporal de natureza GRAVE (art. 129, §1o.)") |  (df_BO_2007a2016['rubrica'] == "Lesão corporal seguida de morte (art. 129, §3o.) 4")]

In [33]:
df_BO_2007a2016_homicidio.columns

Index(['index', 'num_bo', 'ano_bo', 'id_delegacia', 'nome_departamento',
       'nome_seccional', 'delegacia', 'nome_departamento_circ',
       'nome_seccional_circ', 'nome_delegacia_circ', 'ano', 'mes',
       'data_ocorrencia_bo', 'hora_ocorrencia_bo', 'flag_status', 'rubrica',
       'desdobramento', 'conduta', 'latitude', 'longitude', 'cidade',
       'logradouro', 'numero_logradouro', 'flag_status_2', 'descr_tipo_pessoa',
       'cont_pessoa', 'sexo_pessoa', 'idade_pessoa', 'cor', 'descr_profissao',
       'descr_grau_instrucao'],
      dtype='object')

In [34]:
df_BO_2007a2016_homicidio = df_BO_2007a2016_homicidio[['latitude','longitude', 'cidade','logradouro','numero_logradouro','cor', 'descr_profissao','descr_grau_instrucao','idade_pessoa']]
df_BO_2007a2016_lesao = df_BO_2007a2016_lesao[['latitude','longitude', 'cidade','logradouro','numero_logradouro','cor', 'descr_profissao','descr_grau_instrucao','idade_pessoa']]

In [35]:
df_BO_2007a2016_homicidio['homicidio'] = 1

In [36]:
df_BO_2007a2016_homicidio.head()

,latitude,longitude,cidade,logradouro,numero_logradouro,cor,descr_profissao,descr_grau_instrucao,idade_pessoa,homicidio
5324,NaN,NaN,S.PAULO,R BONITOS,86,Parda,DOMESTICA,1 Grau completo,32,1
9487,NaN,NaN,S.PAULO,RUA SEMPRE VIVA,17,NaN,ESTUDANTE,1 Grau completo,16,1
9807,NaN,NaN,ITAQUAQUECETUBA,R SERRA DO MAR,290,NaN,AUTONOMO(A),2 Grau completo,34,1
13003,NaN,NaN,S.PAULO,R CANUTO LUIZ DO NASCIMENTO,87,Parda,ESTUDANTE,1 Grau completo,16,1
13506,NaN,NaN,SUZANO,AV JORGE BEI MALUF,100,NaN,DO LAR,NaN,19,1


In [129]:
df_BO_2007a2016_homicidio['cor'].unique() 

array(['Parda', nan, 'Branca', 'Preta', 'Outros'], dtype=object)

In [130]:
pd.get_dummies(df_BO_2007a2016_homicidio, columns=['cor'])

,latitude,longitude,cidade,logradouro,numero_logradouro,descr_profissao,descr_grau_instrucao,idade_pessoa,homicidio,cor_Branca,cor_Outros,cor_Parda,cor_Preta
5324,NaN,NaN,SÃO PAULO,R BONITOS,86,DOMESTICA,1 Grau completo,32,1,0,0,1,0
9487,NaN,NaN,SÃO PAULO,RUA SEMPRE VIVA,17,ESTUDANTE,1 Grau completo,16,1,0,0,0,0
9807,NaN,NaN,ITAQUAQUECETUBA,R SERRA DO MAR,290,AUTONOMO(A),2 Grau completo,34,1,0,0,0,0
13003,NaN,NaN,SÃO PAULO,R CANUTO LUIZ DO NASCIMENTO,87,ESTUDANTE,1 Grau completo,16,1,0,0,1,0
13506,NaN,NaN,SUZANO,AV JORGE BEI MALUF,100,DO LAR,NaN,19,1,0,0,0,0
14577,NaN,NaN,GUARULHOS,RUA: ALDALIO MARQUES PEREIRA,82,DO LAR,1 Grau incompleto,58,1,0,0,0,0
14784,NaN,NaN,SÃO PAULO,R COMANDAI,11,AUTONOMO(A),NaN,49,1,0,0,1,0
15173,NaN,NaN,SÃO PAULO,R DAVID BEN GURION,203,PROFESSOR(A),2 Grau incompleto,18,1,1,0,0,0
20774,NaN,NaN,SÃO PAULO,R PAULO DE MIRANDA,17,NaN,1 Grau completo,16,1,1,0,0,0
23961,NaN,NaN,SÃO PAULO,R TOME DOMINGOS,300,NaN,NaN,39,1,0,0,1,0


In [40]:
df_BO_2007a2016_homicidio['cidade'].unique() 

array(['SÃO PAULO', 'ITAQUAQUECETUBA', 'SUZANO', 'GUARULHOS', 'CAJAMAR',
       'S.ANDRE', 'FERRAZ DE VASCONCELOS', 'JUQUITIBA', 'CARAPICUIBA',
       'DIADEMA', 'S.ISABEL', 'OSASCO', 'COTIA', 'MAUA', 'BARUERI',
       'TABOAO DA SERRA', 'RIBEIRAO PIRES', 'FRANCO DA ROCHA',
       'MOGI DAS CRUZES', 'S.BERNARDO DO CAMPO', 'EMBU-GUACU',
       'ITAPECERICA DA SERRA', 'SANTANA DE PARNAIBA', 'FRANCISCO MORATO',
       'ITAPEVI', 'S.CAETANO DO SUL', 'CAIEIRAS', 'POA', 'MAIRIPORA',
       'PIRAPORA BOM JESUS', 'BIRITIBA-MIRIM', 'EMBU',
       'RIO GRANDE DA SERRA', 'VARGEM GRANDE PAULISTA', 'JANDIRA',
       'ARUJA'], dtype=object)

In [39]:
df_BO_2007a2016_homicidio.loc[df_BO_2007a2016_homicidio.cidade == 'S.PAULO', 'cidade'] = "SÃO PAULO"

In [41]:
df_BO_2007a2016_homicidio.loc[df_BO_2007a2016_homicidio.cidade == 'S.ANDRE', 'cidade'] = "SANTO ANDRÉ"
df_BO_2007a2016_homicidio.loc[df_BO_2007a2016_homicidio.cidade == 'S.BERNARDO DO CAMPO', 'cidade'] = "SÃO BERNARDO DO CAMPO"

In [43]:
df_BO_2007a2016_homicidio.head(5)

,latitude,longitude,cidade,logradouro,numero_logradouro,cor,descr_profissao,descr_grau_instrucao,idade_pessoa,homicidio
5324,NaN,NaN,SÃO PAULO,R BONITOS,86,Parda,DOMESTICA,1 Grau completo,32,1
9487,NaN,NaN,SÃO PAULO,RUA SEMPRE VIVA,17,NaN,ESTUDANTE,1 Grau completo,16,1
9807,NaN,NaN,ITAQUAQUECETUBA,R SERRA DO MAR,290,NaN,AUTONOMO(A),2 Grau completo,34,1
13003,NaN,NaN,SÃO PAULO,R CANUTO LUIZ DO NASCIMENTO,87,Parda,ESTUDANTE,1 Grau completo,16,1
13506,NaN,NaN,SUZANO,AV JORGE BEI MALUF,100,NaN,DO LAR,NaN,19,1


In [49]:
df_grouped_profissao = df_BO_2007a2016_homicidio[['homicidio','cidade','descr_profissao']].groupby(['cidade','descr_profissao']).count().reset_index()
df_grouped_profissao = df_grouped_profissao.rename(columns={'homicidio': 'homicidio_profissao'})

In [50]:
df_grouped_grau = df_BO_2007a2016_homicidio[['homicidio','cidade','descr_grau_instrucao']].groupby(['cidade','descr_grau_instrucao']).count().reset_index()
df_grouped_grau = df_grouped_grau.rename(columns={'homicidio': 'homicidio_escolaridade'})

In [51]:
#df_grouped_idade = df_BO_2007a2016_homicidio[['homicidio','cidade','idade_pessoa']].groupby(['cidade','idade_pessoa']).count().reset_index()
#df_grouped_idade = df_grouped_idade.rename(columns={'homicidio': 'homicidio_idade'})

In [52]:
df_grouped_mun = df_BO_2007a2016_homicidio[['homicidio','cidade']].groupby(['cidade']).count().reset_index()
df_grouped_mun = df_grouped_mun.rename(columns={'homicidio': 'homicidio_mun'})

In [53]:
df_grouped_mun

,cidade,homicidio_mun
0,ARUJA,1
1,BARUERI,5
2,BIRITIBA-MIRIM,1
3,CAIEIRAS,3
4,CAJAMAR,4
5,CARAPICUIBA,13
6,COTIA,6
7,DIADEMA,21
8,EMBU,4
9,EMBU-GUACU,3


In [56]:
df_joined1 = pd.merge(df_grouped_profissao, df_grouped_grau,  how='inner', on=['cidade'])

In [57]:
df_joined2 = pd.merge(df_grouped_idade, df_joined1,  how='inner', on=['cidade'])

In [58]:
df_joined_final = pd.merge(df_grouped_mun, df_joined2,  how='inner', on=['cidade'])

In [60]:
df_merge_final = df_merge_final.rename(columns={'NomeMunic': 'cidade'})

In [61]:
df_final = pd.merge(df_joined_final, df_merge_final, how='inner', on=['cidade'])

In [62]:
df_final

,cidade,homicidio_mun,idade_pessoa,homicidio_idade,descr_profissao,homicidio_profissao,descr_grau_instrucao,homicidio_escolaridade,idade,Feminino,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres,coeficiente_feminicidio
0,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,0,1.5,SP,0.763,30383.43,1221979,626936,3.2
1,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,1,0.4,SP,0.763,30383.43,1221979,626936,3.2
2,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,2,0.3,SP,0.763,30383.43,1221979,626936,3.2
3,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,3,0.2,SP,0.763,30383.43,1221979,626936,3.2
4,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,4,0.1,SP,0.763,30383.43,1221979,626936,3.2
5,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,5,0.2,SP,0.763,30383.43,1221979,626936,3.2
6,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,6,0.2,SP,0.763,30383.43,1221979,626936,3.2
7,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,7,0.1,SP,0.763,30383.43,1221979,626936,3.2
8,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,8,0.2,SP,0.763,30383.43,1221979,626936,3.2
9,GUARULHOS,53,14,2,APOSENTADO(A),2,1 Grau completo,22,9,0.1,SP,0.763,30383.43,1221979,626936,3.2


In [63]:
df_final.columns

Index(['cidade', 'homicidio_mun', 'idade_pessoa', 'homicidio_idade',
       'descr_profissao', 'homicidio_profissao', 'descr_grau_instrucao',
       'homicidio_escolaridade', 'idade', 'Feminino',
       'Sigla da Unidade da Federação',
       'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)',
       'PIB per capita a preços correntes', 'População residente',
       'População residente - Mulheres', 'coeficiente_feminicidio'],
      dtype='object')

In [66]:
df_final_to_model = df_final[['cidade', 'homicidio_mun',
       'descr_profissao', 'homicidio_profissao', 'descr_grau_instrucao',
       'homicidio_escolaridade',
       'Sigla da Unidade da Federação',
       'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)',
       'PIB per capita a preços correntes', 'População residente',
       'População residente - Mulheres', 'idade', 'Feminino','coeficiente_feminicidio']]

In [67]:
df_final_to_model

,cidade,homicidio_mun,descr_profissao,homicidio_profissao,descr_grau_instrucao,homicidio_escolaridade,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres,idade,Feminino,coeficiente_feminicidio
0,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2
1,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2
2,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2
3,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2
4,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2
5,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,5,0.2,3.2
6,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,6,0.2,3.2
7,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,7,0.1,3.2
8,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,8,0.2,3.2
9,GUARULHOS,53,APOSENTADO(A),2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,9,0.1,3.2


In [68]:
df_final_to_model['descr_profissao'].unique() 

array(['APOSENTADO(A)', 'AUXILIAR DE ENFERMAGEN', 'BALCONISTA',
       'CABELEIREIRO(A)', 'CASEIRO(A)', 'COMERCIANTE', 'DESEMPREGADO(A)',
       'DO LAR', 'DOMESTICA', 'EMPRESARIO(A)', 'ESTUDANTE', 'FEIRANTE',
       'INSTRUMENTADOR CIRURGICO', 'MANICURE',
       'OPERADOR(A) TELEMARKETING', 'PRENDAS DOMESTICAS', 'PROFESSOR(A)',
       'RECEPCIONISTA', 'VENDEDOR(A)', 'CAIXA', 'DIARISTA', 'OPERADOR(A)',
       'SERVENTE', 'AJUDANTE', 'AUXILIAR ADMINISTRATIVO', 'BANCARIO(A)',
       'AUXILIAR DE LIMPEZA', 'ACOUGUEIRO(A)', 'AJUDANTE GERAL',
       'ASSISTENTE ADMINISTRATIVO', 'ASSISTENTE DE VENDAS', 'ATENDENTE',
       'AUTONOMO(A)', 'AUXILIAR DE COZINHA', 'AUXILIAR DE ESCRITÓRIO',
       'AUXILIAR DE LAVANDERIA', 'AUXILIAR DE PRODUCAO',
       'AUXILIAR DE SERVIÇOS GERAIS', 'BABA', 'CAMAREIRO(A)',
       'CHEFE DE SECAO', 'COBRADOR(A)', 'COMISSARIO(A)', 'COPEIRO(A)',
       'COSTUREIRO', 'COSTUREIRO(A)', 'COZINHEIRO(A)', 'DESIGNER',
       'DESOCUPADO(A)', 'EDUCADOR', 'ENFERMEIRO(A)', 'E

In [80]:
df_final_to_model['descr_grau_instrucao'].unique() 

#Analfabeto = 0.142
#1 Grau incompleto = 0.284
#1 Grau completo = 0.426
#2 Grau incompleto = 0.568
#2 Grau completo = 0.71
#Superior incompleto = 0.852
#Superior completo = 1

def ifef(col):
    if col == "Analfabeto":
        return  0.142
    if col == "1 Grau incompleto":
        return 0.284
    if col == "1 Grau completo":
        return 0.426
    if col == "2 Grau incompleto":
        return 0.568
    if col == "2 Grau completo":
        return 0.71
    if col == "Superior incompleto":
        return 0.852
    if col == "Superior completo":
        return 1
    
df_final_to_model_grado = df_final_to_model
df_final_to_model_grado['grau_classif'] = df_final_to_model_grado['descr_grau_instrucao'].apply(ifef)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:
df_final_to_model_grado.columns

Index(['cidade', 'homicidio_mun', 'descr_profissao', 'homicidio_profissao', 'descr_grau_instrucao', 'homicidio_escolaridade', 'Sigla da Unidade da Federação', 'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)', 'PIB per capita a preços correntes', 'População residente', 'População residente - Mulheres', 'idade', 'Feminino', 'coeficiente_feminicidio', 'grau_classif'], dtype='object')

In [85]:
df_final_to_model_grado_ = df_final_to_model_grado.rename(columns={'cidade': 'CIDADE', 
                        'cidade': 'municipio',
                       'homicidio_escolaridade': 'num_homicidio_escolaridade',
                       'grau_classif': 'escolaridade_classif',
                       'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)': 'idhm',
                       'PIB per capita a preços correntes': 'pib',
                       'População residente': 'pop_total',
                       'População residente - Mulheres': 'pop_mulher'})

In [87]:
df_final_to_model_grado_.columns

Index(['municipio', 'homicidio_mun', 'descr_profissao', 'homicidio_profissao', 'descr_grau_instrucao', 'num_homicidio_escolaridade', 'Sigla da Unidade da Federação', 'idhm', 'pib', 'pop_total', 'pop_mulher', 'idade', 'Feminino', 'coeficiente_feminicidio', 'escolaridade_classif'], dtype='object')

In [88]:
df_final_to_model_grado_ = df_final_to_model_grado_[['municipio', 'descr_grau_instrucao', 'num_homicidio_escolaridade', 'Sigla da Unidade da Federação', 'idhm', 'pib', 'pop_total', 'pop_mulher', 'idade', 'Feminino', 'coeficiente_feminicidio', 'escolaridade_classif']]

In [91]:
df_final_to_model_grado_sum = df_final_to_model_grado_[['municipio','coeficiente_feminicidio']].groupby(['coeficiente_feminicidio']).count().reset_index()

In [104]:
df_final_to_model_grado_better = df_final_to_model_grado_
df_final_to_model_grado_better['probMun'] = df_final_to_model_grado_better['coeficiente_feminicidio']/Total

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [99]:
df_final_to_model_grado_better

,municipio,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif,probIdade
0,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426,0.117216
1,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426,0.117216
2,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426,0.117216
3,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426,0.117216
4,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426,0.117216
5,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,5,0.2,3.2,0.426,0.117216
6,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,6,0.2,3.2,0.426,0.117216
7,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,7,0.1,3.2,0.426,0.117216
8,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,8,0.2,3.2,0.426,0.117216
9,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,9,0.1,3.2,0.426,0.117216


In [105]:
df_final_to_model_grado_better['probIdade'] = df_final_to_model_grado_better['Feminino']/100

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
df_final_to_model_grado_better

,municipio,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif,probIdade,probMun
0,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426,0.015,0.117216
1,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426,0.004,0.117216
2,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426,0.003,0.117216
3,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426,0.002,0.117216
4,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426,0.001,0.117216
5,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,5,0.2,3.2,0.426,0.002,0.117216
6,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,6,0.2,3.2,0.426,0.002,0.117216
7,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,7,0.1,3.2,0.426,0.001,0.117216
8,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,8,0.2,3.2,0.426,0.002,0.117216
9,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,9,0.1,3.2,0.426,0.001,0.117216


In [107]:
mun_lat_long = pd.read_excel("s3://g01batalha/MunicipiosBrasil Latitude Longitude.xlsx")

In [108]:
mun_lat_long

,ID,LATITUDE,LONGITUDE,Mun/UF,MUNICIPIO,UF,Valor
0,1,-9.82,-66.88,ACRELANDIA - AC,ACRELANDIA,AC,3.956004e+04
1,2,-10.94,-69.56,ASSIS BRASIL - AC,ASSIS BRASIL,AC,1.784215e+04
2,3,-11.01,-68.74,BRASILEIA - AC,BRASILEIA,AC,9.337339e+04
3,4,-9.83,-67.95,BUJARI - AC,BUJARI,AC,2.851657e+04
4,5,-10.57,-67.67,CAPIXABA - AC,CAPIXABA,AC,1.888646e+04
5,6,-7.63,-72.67,CRUZEIRO DO SUL - AC,CRUZEIRO DO SUL,AC,3.645861e+05
6,7,-11.02,-68.74,EPITACIOLANDIA - AC,EPITACIOLANDIA,AC,6.048974e+04
7,8,-8.16,-70.35,FEIJO - AC,FEIJO,AC,1.406652e+05
8,9,-9.43,-71.88,JORDAO - AC,JORDAO,AC,2.540715e+04
9,10,-7.61,-72.89,MANCIO LIMA - AC,MANCIO LIMA,AC,5.546422e+04


In [111]:
mun_lat_long = mun_lat_long.rename(columns={'MUNICIPIO': 'municipio', 'LATITUDE':'latitude','LONGITUDE':'longitude'})

In [112]:
mun_lat_long.columns

Index(['ID', 'latitude', 'longitude', 'Mun/UF', 'municipio', 'UF', 'Valor'], dtype='object')

In [113]:
mun_lat_long = mun_lat_long[['municipio', 'latitude', 'longitude']]

In [114]:
df_final_to_model_grado_better_geo = pd.merge(mun_lat_long,df_final_to_model_grado_better, on=['municipio'])

In [115]:
df_final_to_model_grado_better_geo

,municipio,latitude,longitude,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif,probIdade,probMun
0,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426,0.015,0.117216
1,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426,0.004,0.117216
2,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426,0.003,0.117216
3,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426,0.002,0.117216
4,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426,0.001,0.117216
5,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,5,0.2,3.2,0.426,0.002,0.117216
6,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,6,0.2,3.2,0.426,0.002,0.117216
7,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,7,0.1,3.2,0.426,0.001,0.117216
8,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,8,0.2,3.2,0.426,0.002,0.117216
9,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,9,0.1,3.2,0.426,0.001,0.117216


In [120]:
df_final_to_model_grado_better_geo['probTot'] = df_final_to_model_grado_better_geo['probIdade'] * df_final_to_model_grado_better_geo['probMun']

In [117]:
df_final_to_model_grado_better_geo.head()

,municipio,latitude,longitude,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif,probIdade,probMun,fatorRisco
0,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426,0.015,0.117216,0.001758
1,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426,0.004,0.117216,0.000469
2,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426,0.003,0.117216,0.000352
3,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426,0.002,0.117216,0.000234
4,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426,0.001,0.117216,0.000117


In [118]:
max_fator = df_final_to_model_grado_better_geo['fatorRisco'].max()

In [119]:
max_fator

0.004219780219780221

In [121]:
df_final_to_model_grado_better_geo['fatorRisco'] = (df_final_to_model_grado_better_geo['probTot'] * 100)/max_fator

In [123]:
df_final_to_model_grado_better_geo.head()

,municipio,latitude,longitude,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif,probIdade,probMun,fatorRisco,probTot
0,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426,0.015,0.117216,41.666667,0.001758
1,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426,0.004,0.117216,11.111111,0.000469
2,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426,0.003,0.117216,8.333333,0.000352
3,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426,0.002,0.117216,5.555556,0.000234
4,GUARULHOS,-23.46,-46.53,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426,0.001,0.117216,2.777778,0.000117


In [124]:
df_final_to_model_grado_better_geo_final = df_final_to_model_grado_better_geo.rename(columns={'fatorRisco': 'factRisco',
                                                                                             'latitude': 'lat',
                                                                                             'longitude': 'long'})

In [125]:
df_final_to_model_grado_better_geo_final.columns

Index(['municipio', 'lat', 'long', 'descr_grau_instrucao', 'num_homicidio_escolaridade', 'Sigla da Unidade da Federação', 'idhm', 'pib', 'pop_total', 'pop_mulher', 'idade', 'Feminino', 'coeficiente_feminicidio', 'escolaridade_classif', 'probIdade', 'probMun', 'factRisco', 'probTot'], dtype='object')

In [127]:
df_final_to_model_grado_better_geo_final = df_final_to_model_grado_better_geo_final[['municipio', 'lat', 'long', 'num_homicidio_escolaridade', 'idhm', 'pib', 'pop_total', 'pop_mulher', 'idade', 'Feminino', 'coeficiente_feminicidio', 'escolaridade_classif', 'probIdade', 'probMun', 'probTot', 'factRisco']]

In [128]:
from io import StringIO
csv_buffer = StringIO()
df_final_to_model_grado_better_geo_final.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('g01batalha', 'df_idade_risco_escolaridade_final.csv').put(Body=csv_buffer.getvalue())

{'ETag': '"b7cc9d5d9173ca9bd1e2b8c963859f62"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sun, 09 Dec 2018 12:59:08 GMT',
   'etag': '"b7cc9d5d9173ca9bd1e2b8c963859f62"',
   'server': 'AmazonS3',
   'x-amz-id-2': '9aw8etGzK83Z9zsFOFBUOq/SDbkUdZ76CNScQbuJLOs09ezoE7xyMdS8oswsRyCTT4jInLUpEuM=',
   'x-amz-request-id': 'DB3AA32911954835'},
  'HTTPStatusCode': 200,
  'HostId': '9aw8etGzK83Z9zsFOFBUOq/SDbkUdZ76CNScQbuJLOs09ezoE7xyMdS8oswsRyCTT4jInLUpEuM=',
  'RequestId': 'DB3AA32911954835',
  'RetryAttempts': 0}}

In [ ]:
df_final_to_model_grado_ = df_final_to_model_grado[['cidade', 'homicidio_mun', 'descr_profissao', 'homicidio_profissao', 'descr_grau_instrucao', 'homicidio_escolaridade', 'Sigla da Unidade da Federação', 'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)', 'PIB per capita a preços correntes', 'População residente', 'População residente - Mulheres', 'idade', 'Feminino', 'coeficiente_feminicidio', 'grau_classif']]

In [90]:
df_final_to_model_grado_.head()

,municipio,descr_grau_instrucao,num_homicidio_escolaridade,Sigla da Unidade da Federação,idhm,pib,pop_total,pop_mulher,idade,Feminino,coeficiente_feminicidio,escolaridade_classif
0,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,0,1.5,3.2,0.426
1,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,1,0.4,3.2,0.426
2,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,2,0.3,3.2,0.426
3,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,3,0.2,3.2,0.426
4,GUARULHOS,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,4,0.1,3.2,0.426


In [ ]:
df_final_to_model_grado_

In [73]:
df_final_to_model_class = pd.get_dummies(df_final_to_model, columns=['descr_profissao'])

In [74]:
df_final_to_model_class.columns

Index(['cidade', 'homicidio_mun', 'homicidio_profissao',
       'descr_grau_instrucao', 'homicidio_escolaridade',
       'Sigla da Unidade da Federação',
       'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)',
       'PIB per capita a preços correntes', 'População residente',
       'População residente - Mulheres', 'idade', 'Feminino',
       'coeficiente_feminicidio', 'descr_profissao_ACOUGUEIRO(A)',
       'descr_profissao_AJUDANTE', 'descr_profissao_AJUDANTE GERAL',
       'descr_profissao_APOSENTADO(A)',
       'descr_profissao_ASSISTENTE ADMINISTRATIVO',
       'descr_profissao_ASSISTENTE DE VENDAS', 'descr_profissao_ATENDENTE',
       'descr_profissao_AUTONOMO(A)',
       'descr_profissao_AUXILIAR ADMINISTRATIVO',
       'descr_profissao_AUXILIAR DE COZINHA',
       'descr_profissao_AUXILIAR DE ENFERMAGEN',
       'descr_profissao_AUXILIAR DE ESCRITÓRIO',
       'descr_profissao_AUXILIAR DE LAVANDERIA',
       'descr_profissao_AUXILIAR DE LIMPEZA',
       'descr_

In [75]:
df_final_to_model_class

,cidade,homicidio_mun,homicidio_profissao,descr_grau_instrucao,homicidio_escolaridade,Sigla da Unidade da Federação,Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010),PIB per capita a preços correntes,População residente,População residente - Mulheres,...,descr_profissao_RECEPCIONISTA,descr_profissao_REPRESENTANTE COMERCIAL,descr_profissao_SECRETARIO(A),descr_profissao_SERVENTE,descr_profissao_TECNICO EM SEG. TRABALHO,descr_profissao_TECNICO(A),descr_profissao_TECNICO(A) EM INFORMATICA,descr_profissao_VENDEDOR(A),descr_profissao_VETERINARIO(A),descr_profissao_VIGILANTE
0,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
1,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
2,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
3,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
4,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
5,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
6,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
7,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
8,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0
9,GUARULHOS,53,2,1 Grau completo,22,SP,0.763,30383.43,1221979,626936,...,0,0,0,0,0,0,0,0,0,0


In [78]:
df_final_to_model_class.columns

Index(['cidade', 'homicidio_mun', 'homicidio_profissao', 'descr_grau_instrucao', 'homicidio_escolaridade', 'Sigla da Unidade da Federação', 'Índice de Desenvolvimento Humano Municipal - 2010 (IDHM 2010)', 'PIB per capita a preços correntes', 'População residente', 'População residente - Mulheres', 'idade', 'Feminino', 'coeficiente_feminicidio', 'descr_profissao_ACOUGUEIRO(A)', 'descr_profissao_AJUDANTE', 'descr_profissao_AJUDANTE GERAL', 'descr_profissao_APOSENTADO(A)', 'descr_profissao_ASSISTENTE ADMINISTRATIVO', 'descr_profissao_ASSISTENTE DE VENDAS', 'descr_profissao_ATENDENTE', 'descr_profissao_AUTONOMO(A)', 'descr_profissao_AUXILIAR ADMINISTRATIVO', 'descr_profissao_AUXILIAR DE COZINHA', 'descr_profissao_AUXILIAR DE ENFERMAGEN', 'descr_profissao_AUXILIAR DE ESCRITÓRIO', 'descr_profissao_AUXILIAR DE LAVANDERIA', 'descr_profissao_AUXILIAR DE LIMPEZA', 'descr_profissao_AUXILIAR DE PRODUCAO', 'descr_profissao_AUXILIAR DE SERVIÇOS GERAIS', 'descr_profissao_BABA',
       'descr_profissa